In [6]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
import dash
import dash_bootstrap_components as dbc
from datetime import datetime
from dash import html
from dash import dcc
import plotly.express as px
import calendar
from dash.dependencies import Input,Output,State

ModuleNotFoundError: No module named 'dash_bootstrap_components'

In [7]:
!pip install dash_bootstrap_components --user

^C


In [ ]:
!pip install dash-bootstrap-components


In [ ]:
# Create a dash application
app = dash.Dash(__name__)
#JupyterDash.infer_jupyter_proxy_config()

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

In [ ]:
#Read Data
df = pd.read_csv('df_worldwide.csv')

In [ ]:
df.info
df.shape

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
update = df['date'].dt.strftime('%m %d, %Y').iloc[-1]


In [ ]:
value = df[df['date'] == df['date'].iloc[-1]]['Confirmed'].sum()
delta = df[df['date'] == df['date'].unique()[-2]]['Confirmed'].sum()

Deaths_value = df[df['date'] == df['date'].iloc[-1]]['Deaths'].sum()
Deaths_delta = df[df['date'] == df['date'].unique()[-2]]['Deaths'].sum()

Recovered_value = df[df['date'] == df['date'].iloc[-1]]['Recovered'].sum()
Recovered_delta = df[df['date'] == df['date'].unique()[-2]]['Recovered'].sum()

Active_value = df[df['date'] == df['date'].iloc[-1]]['Active'].sum()
Active_delta = df[df['date'] == df['date'].unique()[-2]]['Active'].sum()


In [ ]:
df['month'] = df.date.dt.month
MonthsDF = df.groupby(['month']).sum()[['Confirmed','Deaths','Recovered','Active']].reset_index()
country = 'China'
countrysDF = df.groupby(['Country/Region']).sum()[['Confirmed','Deaths','Recovered','Active']].reset_index()

countryDF = countrysDF[countrysDF['Country/Region'] == country] 
countryValues = countryDF[['Confirmed','Deaths','Recovered','Active']]
x = ['Confirmed','Deaths' , 'Recovered','Active']
y = countryValues.values[0]


In [ ]:
countrysDF.sort_values(by = 'Confirmed' , ascending = False , inplace = True)
top5Months =  MonthsDF.sort_values(by = 'Recovered' , ascending = False)
collectedDF = df.groupby(['Country/Region','month']).sum().reset_index()
collectedDF

In [ ]:
month = 5
countryValues = collectedDF[(collectedDF['Country/Region'] == country) & (collectedDF.month == month)]
countryValues = countryValues[['Confirmed','Deaths','Recovered','Active']]
x = ['Confirmed','Deaths' , 'Recovered','Active']
y = countryValues.values[0]


In [ ]:

row = html.Div(
    [
        dbc.Row(
            dbc.Col(
                html.H1(" Covid-19 Tracking "),
                style ={'margin' : '1rem','color':'white' ,'textAlign':'center'},
                
            )
        ),
        dbc.Row(
            [
                dbc.Card(
    [
      dbc.CardHeader("Last Updated on"),
        dbc.CardBody(
            [
                html.H4(update),
                html.H5('end of day')

            ]
        )
    ],
    style={"width": "18rem", 'margin' : '1rem' , 'text-align': 'center' },
),
                dbc.Card(
    [
      dbc.CardHeader(" Cumlative Confirmed Cases "),  
        dbc.CardBody(
            [
                    html.Div(dcc.Graph(id='confirmed',
                       figure={'data': [{'type': 'indicator',
                    'mode': 'number+delta',
                    'value': value,
                    'delta': {'reference': delta,
                              'valueformat': ',g',
                              'relative': False,
                              'increasing': {'color': 'black'},
                              'decreasing': {'color': 'black'},
                              'font': {'size': 25}},
                    'number': {'valueformat': ',',
                              'font': {'size': 40}},
                    'domain': {'y': [0, 1], 'x': [0, 1]}}],
            'layout': go.Layout(
                font=dict(color='blue'),
                height=100
                )}
                      )
        ),
            ]
        )
    ],
    style={"width": "18rem" ,'margin' : '1rem'},
),
                  dbc.Card(
    [  
        dbc.CardHeader(" Recovered  "),
        dbc.CardBody(
            [
                    html.Div(dcc.Graph(id='Recovered',
                       figure={'data': [{'type': 'indicator',
                    'mode': 'number+delta',
                    'value': Recovered_value,
                    'delta': {'reference': Recovered_delta,
                              'valueformat': ',g',
                              'relative': False,
                              'increasing': {'color': 'black'},
                              'decreasing': {'color': 'black'},
                              'font': {'size': 25}},
                    'number': {'valueformat': ',',
                              'font': {'size': 40}},
                    'domain': {'y': [0, 1], 'x': [0, 1]}}],
            'layout': go.Layout(
                font=dict(color='#33d3ab'),
                height=100
                )}
                      )
        ),
            ]
        )
    ],
    style={"width": "18rem",'margin' : '1rem'},
),
                  dbc.Card(
    [
        dbc.CardHeader(" Deaths to Date  "),
        dbc.CardBody(
            [
                
                
                    html.Div(dcc.Graph(id='deaths',
                       figure={'data': [{'type': 'indicator',
                    'mode': 'number+delta',
                    'value': Deaths_value,
                    'delta': {'reference': Deaths_delta,
                              'valueformat': ',g',
                              'relative': False,
                              'increasing': {'color': 'black'},
                              'decreasing': {'color': 'black'},
                              'font': {'size': 25}},
                    'number': {'valueformat': ',',
                              'font': {'size': 40}},
                    'domain': {'y': [0, 1], 'x': [0, 1]}}],
            'layout': go.Layout(
                font=dict(color='red'),
                height=100

                )}
                      )
        ),
            ]
        )
    ],
    style={"width": "18rem" ,'margin' : '1rem'},
)
            ]
        )])

In [ ]:
graphs = html.Div(
    [
        dbc.Row([
            dbc.Col(
        dcc.Graph( 
                        id = 'top10CountryGraph',
                        figure = px.bar(countrysDF.head(10) , x = 'Country/Region', y = 'Confirmed' , title = 'Top 10 Reported Cases by Country')
                    ),style = { 'margin' : '1rem'}),
            dbc.Col(
        dcc.Graph( 
                        id = 'MonthsGraph',
                        figure = px.line(top5Months , x = 'month', y = ['Confirmed','Deaths','Recovered','Active'],title = 'Monthly Reports From All over the World')
                    ),style = { 'margin' : '1rem'})
        
        ])
    ])

In [ ]:
interactiveGraphs = html.Div([
    dcc.Markdown('**Choose Country : ** ', style = {'color':'white' ,'font-size' : '40' , 'margin': '1rem'}),

    dcc.Dropdown(
            id = 'countryDropDown',
        options=[
            { 'label' : country , 'value' : country } for country in df['Country/Region'].unique()
        ],
        value='China'
    ),
    html.Br(),
                    dcc.Graph( 
                        id = 'CountryPermonthGraph',
                        figure = px.bar(x = x,y = y, title = 'Country Statistics' , color = x,labels=dict(x="Case", y="Number of Cases"))
                    ),
    html.Br(),
    dcc.Slider(
        id = 'monthSlider',
        min=min(df.month),
        marks={str(i): '{} 2020'.format(calendar.month_abbr[i]) for i in df.month.unique()},
        value=min(df.month),
        max=max(df.month),
        step=None
    )
])


In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([row , graphs , interactiveGraphs],style = {'background-color' : '#040045'})
@app.callback(
    Output('CountryPermonthGraph' , 'figure'),
    Input('countryDropDown','value'),
    Input('monthSlider' ,'value'))

def updateGraph(country , month):
    countryValues = collectedDF[(collectedDF['Country/Region'] == country) & (collectedDF.month == month)]
    countryValues = countryValues[['Confirmed','Deaths','Recovered','Active']]
    x = ['Confirmed','Deaths' , 'Recovered','Active']
    y = countryValues.values[0]
    return px.bar(x = x,y = y, title = 'Country Statistics per month', color = x,labels=dict(x="Case", y="Number of Cases"))

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

In [ ]:
# fig = px.scatter_mapbox(df,
#                         lat=df.Latitude,
#                         lon=df.Longitude,
#                         hover_name = df['Country/Region'],
#                         hover_data=['Confirmed','Deaths','Recovered' , 'population'],
#                         zoom=0)

# fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()